In [1]:
import json
import csv

# 讀取 JSON
with open("german_verbs_tw_02.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 輸出 CSV
with open("anki_verbs_output.csv", "w", encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # 表頭欄位
    writer.writerow(["德文動詞", "中文意義", "語法特性", "例句（德文）", "例句（中文）", "支配介詞搭配"])

    for vp in data["verb_phrases"]:
        lemma = vp["lemma"]
        original_zh = ', '.join(vp["zh_meanings"].get("original", []))
        contextual_zh = vp["zh_meanings"].get("contextual", "")
        meaning = f"原意：{original_zh}\n語境意義：{contextual_zh}"

        # 語法特性
        grammar = vp.get("grammar", {})
        features = []
        if grammar.get("case_requirement"):
            features.append(f"受詞格位：{grammar['case_requirement']}")
        if grammar.get("is_reflexive"):
            features.append("反身動詞")
        if grammar.get("is_modal"):
            features.append("情態動詞")
        if grammar.get("is_separable_verb"):
            features.append("分離動詞")
        if grammar.get("gov_preposition"):
            features.append(f"支配介詞：{grammar['gov_preposition']}")

        grammar_str = "; ".join(features)

        # 一般例句
        examples = vp.get("examples", [])
        for ex in examples:
            writer.writerow([
                lemma,
                meaning,
                grammar_str,
                ex["de"],
                ex["zh"],
                ""  # 支配介詞例句欄位留空
            ])

        # 支配介詞例句（若有）
        preps = vp.get("prepositional_phrases", [])
        for prep in preps:
            prep_case = prep["case"]
            example = prep["example"]
            writer.writerow([
                lemma,
                meaning,
                grammar_str,
                example["de"],
                example["zh"],
                f"{prep['preposition']} + {prep_case}"
            ])